Crear un notebook que consuma datos en formato JSON en streaming a través de un topic de kafka denominado json_topic. El procesamiento de los streams será utilizando una longitud de ventana de 10 minutos y un intervalo de deslizamiento de 5 minutos. El contenido de los mensajes enviados será como el siguiente: {"nombre":"jose","edad":38,"peso":75.5}

In [2]:
import findspark
findspark.init()

In [6]:
# Importamos librerias
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, FloatType, StringType, StructField
from pyspark.sql.functions import window, col, from_json

In [ ]:
# Arrancamos la session
spark = SparkSession.builder.appName("JSON_Kafka").getOrCreate()

In [ ]:
# Schema JSON

schema = StructType([
    StructField("Nombre",StringType()),
    StructField("Edad",IntegerType()),
    StructField("Peso",FloatType())
])


In [5]:
# Deberemos tener abierto el producer de nuesto sistema Kafka a la hora de inciar nuestro readStream
# Kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic json_topic --create --partitions 3 --replication-factor 1
# Kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic json_topic

In [ ]:
lines = spark\
        .readStream\
        .format("kafka")\ # Especificamos el formato
        .option("kafka.bootstrap.server","localhost:9092")\
        .option("subscribe","json_topic")\# Con la opcion subscribe ponemos el topic de donde obtendremos los datos.
        .option("includeTimestamp","true")\# Habilitaremos la option de timestamp para que capture la hora de ingreso de los datos, usada para la option de ventana
        .load()\
        .selectExpr("CAST(value AS STRING)","timestamp")\# Campo Value es donde se aloja toda la informacion capturada de Kafka
        .select(from_json(col("value"),schema).alias("parsed_value"),"timestamp")\# Aplicamos el esquema 
        .select("parsed_value.*","timestamp")# Leemos solo los campos que nos interesan 

In [ ]:
# Creamos la ventana con un tamaño de 10 minutos y un desplazamiento de 5 minutos y que agrupe por nombre
windowedCounts = lines.groupBy(window(lines.timestamp,"10 minutes","5 minutes"),lines.nombre)\
                        .agg(avg("peso"),max("peso"),min("min"),avg("edad"),max("edad"),min("edad"))

In [ ]:
# La salida sera mediante consola
query = windowedCounts.writeStream\
        .outputMode("complete")\
        .option("truncate","fase")\
        .format("console")\
        .start()
query.awaiTermination()